# 05 Two-way (column or row) relative frequency tables

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.0.1`
import $ivy.`org.apache.spark::spark-mllib:3.0.1`
import $ivy.`org.plotly-scala::plotly-almond:0.8.0`

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                                      

In [3]:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.StringIndexer
import scala.collection.mutable.LinkedHashMap
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
repl.pprinter() = repl.pprinter().copy(defaultHeight = 1)

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import org.apache.spark.sql._

import org.apache.spark.sql.{functions => F}

import org.apache.spark.sql.types._

import org.apache.spark.ml.feature.StringIndexer

import scala.collection.mutable.LinkedHashMap

import org.apache.log4j.{Level, Logger}


[khanacademy](https://www.khanacademy.org/math/ap-statistics/analyzing-categorical-ap/stats-two-way-tables/v/two-way-relative-frequency-tables?modal=1)


![Two-way (column or row) relative frequency tables fig 1](./imgs/01-05-01.png)

In [4]:
val spark = {
  NotebookSparkSession.builder.appName("statistics").master("local").getOrCreate()
}

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@7ed78a5d

In [5]:
import spark.implicits._

import spark.implicits._

In [6]:
val car = Seq.fill(28)("SUV") ++ Seq.fill(35)("Sport car") ++ Seq.fill(97)("SUV") ++ Seq.fill(104)("Sport car")
val accident =  Seq.fill(28)("yes") ++ Seq.fill(35)("yes") ++ Seq.fill(97)("no") ++ Seq.fill(104)("no")
val dataset = LinkedHashMap(
    "Car" -> car,
    "Accident" -> accident
)

car: Seq[String] = ...
accident: Seq[String] = ...
dataset: LinkedHashMap[String, Seq[String]] = ...

In [7]:
val sdf = dataset.values.toList.transpose.map({case List(a: String, b: String) => (a, b)}).toDF(dataset.keys.toList:_*)
sdf.registerTempTable("sdf_table")
sdf.show()

+---+--------+
|Car|Accident|
+---+--------+
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
|SUV|     yes|
+---+--------+
only showing top 20 rows



sdf: DataFrame = [Car: string, Accident: string]

In [8]:
sdf.groupBy("Accident", "Car").count().show()

show at cmd7.sc:1

1 / 1

show at cmd7.sc:1

1 / 1

show at cmd7.sc:1

...

show at cmd7.sc:1

100 / 100

show at cmd7.sc:1

75 / 75

+--------+---------+-----+
|Accident|      Car|count|
+--------+---------+-----+
|      no|      SUV|   97|
|     yes|Sport car|   35|
|      no|Sport car|  104|
|     yes|      SUV|   28|
+--------+---------+-----+



In [9]:
spark.sql("select Accident, Car, count(Accident) as count from sdf_table group by Accident, Car").show()

show at cmd8.sc:1

1 / 1

show at cmd8.sc:1

1 / 1

show at cmd8.sc:1

...

show at cmd8.sc:1

100 / 100

show at cmd8.sc:1

75 / 75

+--------+---------+-----+
|Accident|      Car|count|
+--------+---------+-----+
|      no|      SUV|   97|
|     yes|Sport car|   35|
|      no|Sport car|  104|
|     yes|      SUV|   28|
+--------+---------+-----+



In [10]:
val twoWayTable = sdf.stat.crosstab("Accident", "Car")
twoWayTable.show()

crosstab at cmd9.sc:1

1 / 1

crosstab at cmd9.sc:1

1 / 1

crosstab at cmd9.sc:1

...

100 / 100

crosstab at cmd9.sc:1

75 / 75

+------------+---+---------+
|Accident_Car|SUV|Sport car|
+------------+---+---------+
|         yes| 28|       35|
|          no| 97|      104|
+------------+---+---------+



twoWayTable: DataFrame = [Accident_Car: string, SUV: bigint ... 1 more field]

In [12]:
var freqTable = twoWayTable
freqTable = freqTable.withColumn("SUV", F.col("SUV") / freqTable.select(F.sum("SUV")).collect()(0)(0))
freqTable = freqTable.withColumn("Sport car", F.col("Sport car") / freqTable.select(F.sum("Sport car")).collect()(0)(0))
freqTable.show()

collect at cmd11.sc:4

1 / 1

collect at cmd11.sc:4

1 / 1

collect at cmd11.sc:5

1 / 1

collect at cmd11.sc:5

1 / 1

+------------+-----+------------------+
|Accident_Car|  SUV|         Sport car|
+------------+-----+------------------+
|         yes|0.224|0.2517985611510791|
|          no|0.776|0.7482014388489209|
+------------+-----+------------------+



freqTable: DataFrame = [Accident_Car: string, SUV: double ... 1 more field]